In [25]:
#импорт бибилиотек
import os
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [19]:
## Парсинг данных с сайта "ВашКонтроль"
#примерный алгоритм парсинга данных с сайта "ВашКонтроль", здесь парситсся часть данных, которая была использована в финальном дата сете. 
url = 'https://vashkontrol.ru/reviews/'
firstpage = 2184000 #опытным путем выяснено, что это значение соответствует отзывам, оставленным в январе 2022 года
lastpage = firstpage + 10

#функция перебора страниц
def get_urls():
    listUrl = []
    for page in range(firstpage,lastpage):
        pageUrl = '{}{}'.format(url,page)
        listUrl.append(pageUrl)
        #pageId.append(page)
    return listUrl
len(get_urls())

servList =[]
commentsList = []
authorityList = []
dateList= []
meanScoreList = []
pageId = []


def get_data():
    for pageData in get_urls():
        fullData = requests.get(pageData) #если есть проблемы с сертификатами можно поставить verify = False, хоть это и не рекомендуется)
        if fullData.status_code == 200:
            soup = BeautifulSoup(fullData.text, 'html.parser')
            
            #костыль с проверкой отсутствия отзыва при работающей странице
            if 'Отзыв с такими реквизитами не существует' not in str(soup.find('div', class_="block review")):
                
                #парсинг комментария
                comments = soup.findAll('div',class_= 'block review-comment pull-left span45')
                for comment in range(len(comments)):
                    if comments[comment].find('span',class_ = 'title') is not None:
                        commentsList.append(comments[comment].text.split(':')[1])

                #парсинг названия услуги
                texts = soup.findAll('div',class_= 'text')
                name = str(texts[0]).split(texts[0].findChild("a")['href'])[1].replace('</a></div>','').replace('">','')
                servList.append(name)

                #парсинг ведомства предоставляющего услугу
                #authorities = soup.findAll('div',class_= 'text')
                authority = str(texts[3]).split(texts[3].findChild("a")['href'])[1].replace('</a></div>','').replace('">','')
                authorityList.append(authority)

                #парсинг даты оставления отзыва
                #authorities = soup.findAll('div',class_= 'text')
                date = str(texts[-1].find('span')).replace('<span>','').replace('</span>','')
                dateList.append(date)

                #парсинг оценок
                mList = []
                scores = soup.findAll('div',class_= 'scale')
                #расчет средней оценки
                for score in scores:
                    mList.append(int(score['data-value']))
                meanScoreList.append(round(np.array(mList).mean(),1))

                #добавление страницы в список считанных
                pageId.append(pageData.split('reviews/')[1])
        time.sleep(5.0)

In [ ]:
#вызов функции парсинга данных
get_data()

In [21]:
#создание DataFrame из распарсенных данных
df_raw = pd.DataFrame(data = {'review_id':pageId, 'date':dateList, 'authority': authorityList, 'serv_name':servList,
                         'comment':commentsList, 'mscore':meanScoreList})
#предобработка текста
df_raw = df_raw[df_raw['comment'].notna()]
df_raw['comment'] = df_raw['comment'].str.replace('\n','')
df_raw.replace('None',np.nan, inplace=True)

#экспорт информации в csv
file_path = os.path.join('..', 'data', 'reviews.csv')#путь для выгрузки данных
df_raw.to_csv(file_path,index=False, na_rep=np.nan)